---
title: "Support Vector Machines (SVMs) y optimización de parámetros"
theme: Montpellier
author: "Máster en Ciencias de Datos e Ingeniería de Computadores, Minería de Datos - Preprocesamiento y clasificación"
date: 11/11/2024
date-format: long
toc: true
toc-title: Tabla de Contenidos
toc-depth: 1
execute:
  echo: true
output:
  beamer_presentation:
    slide_level: 1
format:
  html:
    code-fold: false
    code-summary: "Muestra código"
    fig-width: 5
    fig-height: 3
    fig-align: left
  beamer:
    fig-width: 4
    fig-height: 2
  revealjs:
    theme: dark
    fig-align: left
    fig-height: 5
    fig-cap-location: margin
    smaller: true
---

## Support Vector Machines (SVMs)

En esta sesión vamos a repasar los principales métodos de SVMs vistos en teoría. 

Se aplica una evaluación completamente *naif* de los clasificadores, sin ningún esquema de validación. Esta tarea se deja al estudiante para repasar los contenidos de la primera sesión de prácticas.

In [ ]:
#| code-fold: true
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn import datasets
from sklearn.datasets import load_breast_cancer, load_iris
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

breastCancer = datasets.load_breast_cancer()
X_b = breastCancer.data
y_b = breastCancer.target

iris = datasets.load_iris()
X_i = iris.data
y_i = iris.target

# Modelos de SVM

## Distintos tipos de SVM

Hay varios tipos:

- **SVC** es una implementación basada en la famosa libSVM ([https://www.csie.ntu.edu.tw/~cjlin/libsvm/](https://www.csie.ntu.edu.tw/~cjlin/libsvm/)) y, en particular, la versión con el factor C que permite errores a la hora de buscar el margen.

- SVC: El modelo general.
- LinearSVC: Implementa solo el lineal, más eficiente y algunas regularizaciones adicionales. Usa la implementación de liblinear: [https://www.csie.ntu.edu.tw/~cjlin/liblinear/](https://www.csie.ntu.edu.tw/~cjlin/liblinear/).
- NuSVC: Como el SVC, pero utiliza parámetro nu (ratio de margen de error).

El nombre SVC se refiere a clasificación, para regresión es SVR.

# Funciones sintéticas

## Con funciones sintéticas 

Primero vamos a definir distintos datasets sintéticos, y un modelo por kernel.

In [ ]:
#Hacemos un problema fácil de clasificar y que sea linealmente separable
X, y = make_classification(
    n_features=2, n_redundant=0, n_informative=2, random_state=0, n_clusters_per_class=1
)
rng = np.random.RandomState(2)
X += 2 * rng.uniform(size=X.shape)
linearly_separable = (X, y)

datasets = [
    make_moons(noise=0.3, random_state=0),
    make_circles(noise=0.2, factor=0.5, random_state=0),
    linearly_separable
]

models = {'linear': SVC(kernel='linear', C=1.0, random_state=0),
          'poly': SVC(kernel='poly', degree=3, C=1.0, gamma=1,random_state=0),
          'rbf': SVC(kernel='rbf', C=1.0, gamma=1,random_state=0)}

---

Ahora vamos a pintar para cada dataset la gráfica. Primero el código.

Hago uso de yellowbrick para pintar las regiones de decisión, usando una subfigura por conjunto.

In [ ]:
from yellowbrick.contrib.classifier import DecisionViz


def pinta_regiones_decision_dataset(datasets, model):
    fig, axs = plt.subplots(1, len(datasets), figsize=(12,6))
    for (i, dataset) in enumerate(datasets):
        viz = DecisionViz(model, ax=axs[i])
        X, y = dataset
        X = StandardScaler().fit_transform(X)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.4, random_state=42
        )
        viz.fit(X_train, y_train)
        viz.draw(X_test, y_test)
        viz.finalize()

    plt.show()

---

Probamos un enfoque lineal:

In [ ]:
pinta_regiones_decision_dataset(datasets, models['linear'])

--- 

Ahora un enfoque kernel _polinomial_:

In [ ]:
pinta_regiones_decision_dataset(datasets, models['poly'])

--- 

Ahora un enfoque kernel _rbf_:

In [ ]:
pinta_regiones_decision_dataset(datasets, models['rbf'])

--- 

Ahora vamos a mostrar el comportamiento de cada uno de los modelos anteriores, usando cross_validation.

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_validate, cross_val_score


def pinta_regiones_decision_model(dataset, models):
    fig, axs = plt.subplots(1, len(models), figsize=(12,6))
    for (i, model) in enumerate(models):
        viz = DecisionViz(model, ax=axs[i])
        X, y = dataset
        X = StandardScaler().fit_transform(X)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.4, random_state=42
        )
        viz.fit(X_train, y_train)
        viz.draw(X_test, y_test)
        viz.finalize()

    plt.show()

def muestra_error_region(dataset, models):
    for (name, model) in models.items():
        X_d, y_d = dataset
        scores = cross_validate(model, X_d, y_d, cv=5, scoring=('accuracy', 'roc_auc'))
        accu = np.mean(scores['test_accuracy'])
        print(f"{name}: {accu:.2f}", end="\t")

    return pinta_regiones_decision_model(dataset, models.values())

---

Primer dataset: 

In [ ]:
muestra_error_region(datasets[0], models)

---

Segundo dataset: 

In [ ]:
muestra_error_region(datasets[1], models)

---

Tercer dataset: 

In [ ]:
muestra_error_region(datasets[2], models)

# datasets

## Sobre los datasets

Vamos a ajustar sobre los conjuntos de entrenamiento de juguete de sklearn. Empezamos con Iris, que tiene atributos numéricos y, por tanto, no necesita hacer modificaciones en los atributos de entrada.


In [ ]:
# Vamos a empezar con kernel lineal sencillo
svm = SVC(kernel='linear', C=1.0, random_state=0)

svm.fit(X_i, y_i)
y_pred = svm.predict(X_i)
print("Informe completo\n",classification_report(y_i, y_pred))

---

Si queremos ajustar sobre el conjunto de *breast_cancer*, nos encontramos con el problema de tener atributos nominales.

Por defecto, se convertirán los atributos nominales a índices enteros, que sirven para el producto escalar, pero introducen un significado de magnitud relativa entre los valores nominales no existe en la realidad, pudiendo producir importantes problemas de rendimiento.

In [ ]:
svm = SVC(kernel='linear', C=1.0, random_state=0)

svm.fit(X_b, y_b)
y_pred = svm.predict(X_b)
print("Informe completo\n",classification_report(y_b, y_pred))

---

Para solventar este problema, podemos utilizar *OneHotEncoding*, que transforma los atributos nominales en conjuntos de atributos binarios que sí que mantienen el sentido original de los valores categóricos al hacer el producto escalar en el Kernel.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot = OneHotEncoder(handle_unknown='ignore') # Los valores perdidos se eliminarán

X_b_onehot = one_hot.fit_transform(X_b) # ajustamos la transformación para crear los atributos (recuerda la regresión polinómica donde se introducían nuevos atributos!)

#Vamos a comparar los atributos de entrada de los dos problemas
print(X_b.shape)
print(X_b_onehot.shape)

svm = SVC(kernel='linear', C=1.0, random_state=0)

svm.fit(X_b_onehot, y_b)
y_pred = svm.predict(X_b_onehot)
print("Informe completo\n",classification_report(y_b, y_pred)) #Compara los resultados con el anterior en el que no usábamos one-hot encoding

---

In [ ]:
# Vamos a empezar con kernel lineal sencillo
# nu es el parámetro de regularización, si lo ponemos muy alto, el modelo se ajustará mucho a los datos de entrenamiento
# o directamente dará error porque no puede conseguir ajustarse a ese valor
nusvm = NuSVC(kernel='linear', nu=0.5, random_state=0) 

nusvm.fit(X_i, y_i)
y_pred = nusvm.predict(X_i)
print("Informe completo NuSVM\n",classification_report(y_i, y_pred))

# Problemas con múltiples clases

## Estrategias de división del conjunto de entrenamiento para clasificadores binarios

**!Un momento!** Iris es un conjunto con 3 clases. ¿Cómo está consiguiendo la SVM clasificar más de 2 etiquetas?
Por defecto, SVC de Sklearn implementa *One versus Rest (OVR)* ó *One versus All (OVA)* como estrategia de división: se generarán subconjuntos a partir del conjunto de entrenamiento original, enfrentando cada clase frente a las otras combinadas.

---

Esta estrategia se diferencia *One versus All (OVO)*, que enfrenta cada clase frente a una del resto.

![OVO y OVA](ovo_ovr.png){width="70%"}

Vamos a utilizar el *wrapper* de Sklearn para generar el mismo efecto en SVC para simular el efecto de OVO y  comprobar que los efectos son similares.

---

In [ ]:
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

svm_lineal = LinearSVC(C=1.0, random_state=0);

svm_lineal.fit(X_i, y_i)
y_pred = svm_lineal.predict(X_i)
print("Informe completo para OVR incorporado\n",classification_report(y_i, y_pred))

ovr = OneVsRestClassifier(LinearSVC(C=1.0,random_state=0)); # Usamos mismos parámetros que antes y la misma semilla
ovr.fit(X_i, y_i)
y_pred = ovr.predict(X_i)
print("Informe completo para OVR wrapper\n",classification_report(y_i, y_pred))

---

Ahora vamos a probar a utilizar el wrapper para utilizar **OVO** y observamos las diferencias.

In [ ]:
from sklearn.multiclass import OneVsOneClassifier
#Para comparar fácilmente, recuperamos el último informe previo
print("Informe completo para OVR wrapper\n",classification_report(y_i, y_pred))

ovo = OneVsOneClassifier(LinearSVC(C=1.0,random_state=0)); # Usamos mismos parámetros que antes y la misma semilla
ovo.fit(X_i, y_i)
y_pred = ovo.predict(X_i)
print("Informe completo para OVO wrapper\n",classification_report(y_i, y_pred))

---

Por norma general, si el número de clases **no** es muy alto, la estrategia OVO es preferible. Genera problemas menos desequilibrados y más pequeños, que son más fáciles de tratar por el clasificador (y genera modelos más sencillos). Además, OVO es preferible en problemas desequilibrados originariamente, ya que no acentuará el desequilibrio como si acabará haciendo OVA.

No obstante, la combinatoria juega en nuestra contra con OVO, por lo que ante un número de clases alto la estrategia OVA (o One vs. Rest) puede ser necesaria.

## Ejercicios

1. Probar para los problemas sintéticos cambios en los parámetros de los modelos para aplicar. Ver los cambios tanto en la gráfica como en las métricas.

2. Probar con otro modelo de IA el uso de OVO y OVR.